# CBR from scratch

To do that, we work on the [Palmer Penguins dataset](https://allisonhorst.github.io/palmerpenguins/).

## The Libraries

We use `pandas` for table/DataFrame manipulation, `numpy` for array algebra and the `spatial.distance` module of `scipy` for our test similarity metric: the `euclidean` distance.

In [1]:
import pandas as pd
import numpy as np
import palmerpenguins as pengu
from scipy.spatial.distance import euclidean

## The Case base

We load the case base and take a peek:

In [2]:
casebase = pengu.load_penguins()
casebase.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [18]:
casebase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
 7   year               344 non-null    int64  
dtypes: float64(4), int64(1), object(3)
memory usage: 21.6+ KB


To ensure that we can use _any_ similarity metric, we can generalise.
I like to think that when we are calculating `dist` between `u` and `v`, we are just _applying_ a similarity metric $f$ to such cases $u$ and $v$:
$$
\text{dist}_f(u, v) = f(u, v), \quad f \in F
$$
where $f$ is a similarity metric in the set of all similarity metrics $F$.

In [3]:
def dist(f:callable, u:np.ndarray, v:np.ndarray) -> float:
    return f(u, v)

With this generalisation, we can use any distance metric that is already coded as a _callable_: any _function_ in programming terms.

## An example: $f(u,v)$

Let's try a simple case. Let's consider penguin #42 (for no particular reason) and look at its attributes:

In [4]:
casebase.iloc[42]

species              Adelie
island                Dream
bill_length_mm         36.0
bill_depth_mm          18.5
flipper_length_mm     186.0
body_mass_g          3100.0
sex                  female
year                   2007
Name: 42, dtype: object

For the sake of simplicity, let's assume that we will work only with the numerical attributes.
Here are the numerical attributes:

In [5]:
case1 = casebase.iloc[42, 2:6]
case1

bill_length_mm         36.0
bill_depth_mm          18.5
flipper_length_mm     186.0
body_mass_g          3100.0
Name: 42, dtype: object

Let's get the numerical attributes of another individual, penguin #69:

In [6]:
case2 = casebase.iloc[69, 2:6]
case2

bill_length_mm         41.8
bill_depth_mm          19.4
flipper_length_mm     198.0
body_mass_g          4450.0
Name: 69, dtype: object

We can then use our `dist` function by providing our similarity metric and both cases:

In [7]:
dist(euclidean, case1, case2)

1350.0660909748085

This is a _distance metric_: the `euclidean`` distance between the two cases.

## An example: $k$-most similar cases

Now let's assume that we want to find the most similar cases to our `case1`, penguin #42.
To do that, we have to compute the similarity between our case and every other case on the case base.

In [8]:
precomputed_distances = casebase.drop(index=42).iloc[:, 2:6].apply(lambda x: dist(euclidean, case1, x), axis=1)
precomputed_distances

0       650.026653
1       700.009614
2       150.332099
3              NaN
4       350.071607
          ...     
339     900.463619
340     300.520232
341     675.173348
342    1000.397566
343     675.256011
Length: 343, dtype: float64

This is a one-liner which might not be the best for clarity, but is quite compact. This is what we are doing:

1. Drop individual #42 from the case base
2. Get all the numeric values (i.e., columns 2 to 6 (non-inclusive))
3. And then apply a function $dist(euclidean, case_1, x)$ for all $x$ `casebase`'s rows

The result is a matrix of precomputed distances in which we need to search for the $k$ _smaller_ elements because this is a distance metric, which we want to minimise.

When we set $k=1$, what we have is the closest individual. This is equivalent to the $\argmin$ of the `precomputed_distances`:

In [9]:
precomputed_distances.argmin()

101

The $\argmin$ function returns the **index** of _the individual with the minimum distance_. So we need to fetch it:

Which means index 101 is the nearest. Let's look at its features:

In [10]:
casebase.iloc[101]

species              Adelie
island               Biscoe
bill_length_mm         41.0
bill_depth_mm          20.0
flipper_length_mm     203.0
body_mass_g          4725.0
sex                    male
year                   2009
Name: 101, dtype: object

And here is our test case, individual #42, for comparison:

In [11]:
casebase.iloc[42]

species              Adelie
island                Dream
bill_length_mm         36.0
bill_depth_mm          18.5
flipper_length_mm     186.0
body_mass_g          3100.0
sex                  female
year                   2007
Name: 42, dtype: object

They're both the same species so that's nice!

We can do the same, and depending on the size of the case base either: a) find the $k$-min or b) sort `casebase`` and get the first $k$ elements.
Let's be efficient and do the second:

In [12]:
np.argpartition(precomputed_distances, 5)[:5]

array([119, 101,  59,  40, 123])

Which correponds to the _first 5 elements_ of a partially sorted array.
If we check the `species` of each of the five most similar cases, we have the following:

In [13]:
for i in np.argpartition(precomputed_distances, 5)[:5]:
    print(casebase.iloc[i].species)


Adelie
Adelie
Adelie
Adelie
Adelie


All of them are of the same species. Who would have guessed, huh?

This checks out: penguins of the same species would have similar characteristics $\blacksquare$